In [4]:
import pandas as pd
import streamlit as st
import graphviz


# TODO: How do we make this in front (beyond the "put it in render")?


df = pd.DataFrame(
    [['this', 'func', 'that'], ['The, quick', 'brown', 'fox, jumps']],
    columns=["Outputs", "Function", "Inputs"],
)




def df_to_triples(df):
    for row in df.itertuples():
        yield row.Outputs, row.Function, row.Inputs


def triple_to_edges(triple_gen):
    for outputs, func, inputs in triple_gen:
    #outputs, func, inputs = triple
        func = func.strip()
        for output in outputs.split(','):
            for input in inputs.split(','):
                yield output.strip(), func
                yield func, input.strip()


from dol import Pipe

row_to_edges = Pipe(df_to_triples, triple_to_edges)
list(df_to_triples(df))

[('this', 'func', 'that'), ('The, quick', 'brown', 'fox, jumps')]

In [6]:
list(row_to_edges(df))

[('this', 'func'),
 ('func', 'that'),
 ('The', 'brown'),
 ('brown', 'fox'),
 ('The', 'brown'),
 ('brown', 'jumps'),
 ('quick', 'brown'),
 ('brown', 'fox'),
 ('quick', 'brown'),
 ('brown', 'jumps')]